In [1]:
import daphmeIO

In [13]:
loader = daphmeIO.DataLoader()

In [14]:
loader.load_spark("s3://phl-pings/gravy/")

In [20]:
df = loader.df

In [ ]:
df.select('uid').distinct().show()

In [21]:
df = df.selectExpr("grid as uid")
df.show()

+--------------------+
|                 uid|
+--------------------+
|00000006-23aa-3e4...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
|00000013-ec62-3ef...|
+--------------------+
only showing top 20 rows



# Test formats and folder structures

## Generate test dataset

In [11]:
import pandas as pd

In [12]:
path = "s3://synthetic-raw-data/100-agents/sparse_trajectories.parquet"
df = pd.read_parquet(path)

In [13]:
df

,x,y,local_timestamp,unix_timestamp,identifier,hor_accuracy
unix_timestamp,,,,,,
1704119340,16.960116,9.179240,2024-01-01 14:29:00,1704119340,wizardly_joliot,0.011439
1704119700,16.695553,8.892945,2024-01-01 14:35:00,1704119700,wizardly_joliot,0.127361
1704155880,11.374734,14.598104,2024-01-02 00:38:00,1704155880,wizardly_joliot,0.190629
1704156000,11.207716,14.686354,2024-01-02 00:40:00,1704156000,wizardly_joliot,0.156493
1704156840,11.114222,14.608686,2024-01-02 00:54:00,1704156840,wizardly_joliot,0.113733
...,...,...,...,...,...,...
1705303380,7.222383,8.214435,2024-01-15 07:23:00,1705303380,angry_spence,0.189583
1705303740,7.325798,7.928358,2024-01-15 07:29:00,1705303740,angry_spence,0.115442
1705303980,7.110549,8.060382,2024-01-15 07:33:00,1705303980,angry_spence,0.168213


In [66]:
import pyarrow.parquet as pq
import pyarrow.fs as fs
import pyarrow.dataset as ds
import pyarrow as pa

s3 = fs.S3FileSystem()

path = "s3://synthetic-raw-data/100-agents/sparse_trajectories.parquet"
dataset = ds.dataset(path[5:], format="parquet", filesystem=s3)
df = dataset.to_table().to_pandas()

In [68]:
df['date'] = pd.to_datetime(df['timestamp'], unit='s').dt.date

In [69]:
table = pa.Table.from_pandas(df)

In [76]:
path = "../data/sample3/"
dataset = ds.dataset(path, format="parquet", partitioning='hive')
df = dataset.to_table().to_pandas()

In [77]:
df

,uid,timestamp,latitude,longitude,date
0,wizardly_joliot,1704119340,38.321711,-36.667334,2024-01-01
1,wizardly_joliot,1704119700,38.321676,-36.667365,2024-01-01
0,wonderful_swirles,1704121560,38.321017,-36.667869,2024-01-01
0,youthful_galileo,1704098820,38.321625,-36.666612,2024-01-01
1,youthful_galileo,1704103140,38.321681,-36.666841,2024-01-01
...,...,...,...,...,...
1691,angry_spence,1705303380,38.320399,-36.667438,2024-01-15
1692,angry_spence,1705303740,38.320413,-36.667469,2024-01-15
1693,angry_spence,1705303980,38.320384,-36.667455,2024-01-15
1694,angry_spence,1705304340,38.320349,-36.667473,2024-01-15
